In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

In [2]:
# replacement={
#     '83/83':'1',    
# }

# # Replacement
# multi[curation_col]= multi[curation_col].apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

# matches=not_tube
# matches[curation_col]=matches['height'].astype(str)
# matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'7\/165','.44',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))


# string = range_string
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return f'{lower}" - {upper}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return f'{original_val}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

# matches[curation_col]=matches['Min'].apply(range_app)
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))  

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]  

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))    

def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        print(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

# str(round(num, 2))
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers

def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers

def deci(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

In [4]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = """Multiple Buckets"""
strategy_version_input=682
attribute_id_input=2426
formatted_attribute = 'width'

In [5]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [6]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]

In [8]:
buckets

[7067,
 7448,
 7475,
 8829,
 7444,
 7529,
 8681,
 7467,
 7094,
 7432,
 7535,
 9582,
 7479,
 7474,
 7421,
 7491,
 7484,
 8857,
 7222,
 7438,
 8110,
 7461,
 7089,
 8612,
 7428,
 8751,
 6781,
 7443,
 7831,
 8598,
 6876,
 7476,
 8592,
 7259,
 7429,
 7430,
 7471,
 7488,
 7477,
 7447,
 8956,
 7446,
 6868,
 7472,
 8709,
 9591,
 7492,
 7451,
 9733,
 7490,
 7455,
 9095,
 7440,
 7481,
 7466,
 9244,
 7422,
 7442,
 9594,
 8680,
 7520]

In [14]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
print(len(df))
df = df[df['bucket_id'].isin(buckets)]
df['bucket_id'].explode().value_counts()

17893


7467    2
Name: bucket_id, dtype: int64

In [15]:
df['buckets'].explode().value_counts()[0:5]

Cabinet Accessories    2
Name: buckets, dtype: int64

In [16]:
df

product_id external_id                            product_name  \
10404  1173647677       26798            5/16 Round Bump-On Protector   
10405  1173647676       26800  7/8 Depressed Center Bump-On Protector   

                                                                                                                                                                                                                                                                                                                                                                                        long_desc  \
10404  Molded polyurethane shapes with a pressure sensitive adhesive backing. Ideal as replacement "feet" for test equipment. Use as bumpers for doors, hoods, cabinets and other applications. Provides high vibration and shock resistance. Excellent slip resistance and will not mar other surfaces. High tack pressure sensitive adhesive holds securely. Will not crack or harden with age.   
10405  Molded polyurethane shapes with a pressure sensitive adhesive backing. Ideal as replacement "feet" for test equipment. Use as bumpers for doors, hoods, cabinets and other applications. Provides high vibration and shock resistance. Excellent slip resistance and will not mar other surfaces. High tack pressure sensitive adhesive holds securely. Will not crack or harden with age.   

       customer_id attributes              buckets  bucket_id values  
10404          137       None  Cabinet Accessories       7467   None  
10405          137       None  Cabinet Accessories       7467   None

In [57]:
regex_pattern=r'(?i)(\d+.?\d*\/\d*)|()'
height['matches']=height['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
total_na=height[height['matches'].astype(str)=='[]']
regex_pattern_total=r'(?i)(\d+.?\d*\"?.?\s?\s?x)|(\d+.?\d*\"?.?\s?\s?.{0:100}?w)|(\d+\"?\s?.?.?.?.?.?.?.?.?.?.?.?.?.?.?.?w)|()'
total_na['new_matches']=total_na['long_desc'].apply(lambda x: re_extract(regex_pattern_total, x))
total_na_values=total_na[total_na['new_matches'].astype(str)=='[]']
print(len(total_na_values))
total_na_values[curation_col]='n/a'

1075


In [58]:
total_na_values[0:500]

product_id external_id  \
3      519865867  5637516789   
5     1263048038  5637587860   
22    1263048046  5637587867   
24     517344610  5637514939   
31     434491051  5637465480   
35     492030506  5637508324   
42     434422160  5637353906   
51     434510632  5637489550   
52     434513511  5637490700   
57     434496605  5637471345   
59     434485478  5637171376   
65    1263048037  5637587859   
73     434385877  5637217737   
107   1181249625  5637547626   
110   1263048035  5637587858   
111   1263048034  5637587857   
114    434426596  5637355356   
127    434510631  5637489549   
130    434510597  5637489517   
131    434505607  5637489454   
139   1175427861  5637576121   
141    536514443  5637521285   
145    434445068  5637397319   
157    492030432  5637508249   
165   1195683895  5637472993   
168    492030433  5637508250   
174    492030437  5637508254   
178    492030406  5637508268   
187    492030435  5637508252   
189    528382241  5637472991   
199    434496604  5637471344   
210    434496609  5637471348   
230    434513893  5637492360   
244    434391935  5637246229   
253    434487403  5637164750   
254    434485772  5637149769   
259    434415098  5637328791   
261    434438922  5637378472   
268    434437501  5637381985   
271    434388071  5637240795   
273    434415071  5637328734   
274    492030577  5637508395   
275    492030498  5637508316   
277    434445071  5637397322   
286    434496858  5637472685   
293    434444288  5637399273   
295    434513903  5637492370   
304    546023774  5637523017   
306    815177545  5637541630   
307    434496612  5637471351   
308    434496613  5637471352   
309    547601340  5637523365   
314    492030501  5637508319   
335    434389814  5637241132   
337    815179440  5637561783   
348    434509551  5637490805   
349    434510707  5637489625   
351    434509548  5637490802   
358    434508654  5637490182   
363    434477872  5637145143   
367    434501445  5637472995   
371    517345051  5637514896   
374    434440698  5637392776   
378    434513551  5637490739   
386    434496638  5637471377   
389    546023769  5637523012   
393    434497675  5637473017   
394    776730194  5637556834   
395    776730192  5637556832   
420    686685980  5637547706   
421    686685975  5637547701   
422    686685974  5637547700   
424    434390695  5637241615   
425    492030504  5637508322   
428    777267734  5637472989   
440    791130358  5637559871   
449    517346395  5637514986   
491   1175394907  5637576682   
492   1175394905  5637576680   
495    434436041  5637372196   
531    434513891  5637492358   
560   1175427867  5637576142   
628    434504207  5637485868   
629    434504206  5637485867   
630    434504204  5637485864   
631    434504203  5637485863   
632    434504202  5637485862   
647    434508395  5637492007   
648   1175444812  5637576096   
655    434500174  5637478932   
656    434500173  5637478931   
657    434500172  5637478930   
688    434498387  5637470877   
689    434433400  5637370409   
694    434498384  5637470874   
697    434498383  5637470873   
725    434498376  5637470866   
727    434388111  5637241368   
728    434496029  5637471981   
951    434411618  5637312362   
952    434411615  5637312359   
959    434411531  5637311992   
1012   434496637  5637471376   
1017   434496622  5637471361   
1018   434496621  5637471360   
1019   434496614  5637471353   
1020   434496611  5637471350   
1021   434496610  5637471349   
1051   434400922  5637283705   
1065   434470173  5637433843   
1067   434470165  5637433837   
1144   434509549  5637490803   
1145   434509547  5637490801   
1196   546786448  5637523787   
1199   434477755  5637155587   
1232   434479622  5637455291   
1243   434387349  5637217273   
1251   434401614  5637280799   
1260   434505843  5637500901   
1261   434505842  5637500900   
1267   434516805  5637500871   
1268   434516803  5637500869   
1316   434467849  5637432885   
1319   434466029  5637432878   
1320  

# General

In [9]:

# regex_pattern_general=r'(?i)(\b\d+\.\d*.?\"\sH)|(\b\d+.?\"\sH)|(\b\d+\.\d*.?\"\sH)|(\d*\s?\-\d*\/\d*\\\"\sH)|(\d*\s\-?\d*\/\d*\\\"\sH)|(\d+\.?\d*''\sH)|()'
# # regex_pattern_general=r'(?i)(\b\d+\.\d*.?\"\s?H)|(\b\d+.?\"\s?H)|(\b\d+\.\d*.?\"\s?H)|(\d*\s?\-\d*\/\d*\\\"\s?H)|(\d*\s\-?\d*\/\d*\\\"\s?H)|(\d+\.?\d*''\s?H)|()'
# height['matches'] = height['long_desc'].apply(lambda x: re_extract(regex_pattern_general, x))
# height['name_matches'] = height['product_name'].apply(lambda x: re_extract(regex_pattern_general, x))
# height['matches']= height['matches'].apply(lambda x: sorted(x, key=natural_key))
# print(len(height))
# print(len(height[height['matches'].astype(str)=='[]']))
# print(len(height[height['matches'].astype(str)!='[]']))

In [10]:
# # 3
# (\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|

# # All
# (?i)(\d\d\.?\d*\s?h)|(\d+\s?\d*\/\d*\s?h)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|(\d+\.?\s?\-?\d*\/?\d*"H)|(height\s?without\s?casters.?\s?\d+\")

In [11]:
labels=height[height['buckets'].astype(str)=='Labels, Stickers & Tickets']

In [12]:
tubes_new=labels
ft=r'(?i)(\d+.?\d*\/?\d*.?\s?x\s?\d+.?\d*\/?\d*\s?ft)|()'              
tubes_new['matches']=tubes_new['product_name'].apply(lambda x: re_extract(ft, x))
tubes_new=tubes_new[tubes_new['matches'].astype(str)=='[]']
regex_pattern_double=r'(?i)(\d+.?\d*\/?\d*.?\s?x\s?\d+.?\d*\/?\d*)|()'              
tubes_new['matches']=tubes_new['product_name'].apply(lambda x: re_extract(regex_pattern_double, x))
tubes_new=tubes_new[tubes_new['matches'].astype(str)!='[]']
len(tubes_new)

2969

In [13]:
tubes_new['matches']=tubes_new['matches'].str.get(0).apply(lambda x: x.lower())
tubes_new[['height','width']]=tubes_new['matches'].str.split("x",expand=True,)
tubes_new['height']=tubes_new['height'].apply(lambda measure: re.sub(r'7 200','7',str(measure)))
tubes_new['height']=tubes_new['height'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•','',str(measure)))
tubes_new['height']=tubes_new['height'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))
tubes_new['height']=tubes_new['height'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
tubes_new['height']=tubes_new['height'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
tubes_new['height_new'] = tubes_new['height'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)

In [14]:
tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))

In [15]:
tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_frac(x))
tubes_new['height_new']=tubes_new['height_new'].astype(float)
TWOPLACES = Decimal(10) ** -2
tubes_new['height_new']=tubes_new['height_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
tubes_new[curation_col]=tubes_new['height_new']
tubes_new[curation_col]=tubes_new[curation_col].apply(lambda x: str(x)+' in')

In [16]:
tubes_new

product_id external_id  \
396     434500365  5637480081   
397     434500361  5637480077   
398     434500360  5637480076   
399     434500358  5637480074   
470     525242041  5637518708   
...           ...         ...   
14158   434429300  5637369375   
14160   434481070  5637147794   
14161   434481016  5637147518   
14162   434463144  5637147671   
14163   434463169  5637147676   

                                                                                            product_name  \
396                         W.B. Mason Co. Plastic Label Holder Insert Cards, 2 7/8" x 8", White, 150/CS   
397                         W.B. Mason Co. Plastic Label Holder Insert Cards, 1 1/4" x 3", White, 800/CS   
398                         W.B. Mason Co. Plastic Label Holder Insert Cards, 1 1/8" x 8", White, 400/CS   
399                             W.B. Mason Co. Plastic Label Holder Insert Cards, 1" x 6", White, 500/CS   
470                             W.B. Mason Auto Supplies Slogan, 4 X 4, Fluorescent Green & Black, 12/PK   
...                                                                                                  ...   
14158                                 Avery UltraDuty Warning Labels, 11" x 8.50",  White, 500 Labels/BX   
14160                    Avery Easy Peel� Address Labels, Permanent Adhesive, Clear, 1 1/3" x 4", 350/PK   
14161                                Avery Permanent Durable ID Laser Labels, 2 x 2-5/8, White, 750/Pack   
14162  Avery Internet Shipping Labels, TrueBlock� Technology, Permanent Adhesive, 5 1/2" x 8 1/2", 50/PK   
14163     Avery Shipping Labels, Inkjet, TrueBlock� Technology, Permanent Adhesive, 3 1/3" x 4",  150/PK   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          long_desc  \
396                                                                                                                                                                                                                                                                                                                                                               Plastic Label Holder Insert Cards, 2 7/8" x 8", White, 150/CS•Insert cards sized to fit plastic label holders•8 1/2 x 11" letter sized perforated sheets•High quality white laser paper•Compatible with laser and inkjet printers   
397                                                                                                                                                                                                                                                                                                                                                               Plastic Label Holder Insert Cards, 1 1/4" x 3", White, 800/CS•Insert cards sized to fit plastic label holders•8 1/2 x 11" letter sized perforated sheets•High quality white laser paper•Compatible with laser and inkjet printers   
398                                                                                                                                                                                                                                                                                                                                                               Plastic Label Holder Insert Cards, 1 1/8" x 8", White, 400/CS•Insert cards sized to fit plastic label holders•8 1/2 x 11" letter sized perforated sheets•High quality white laser paper•Compatible with laser and 

# Protective Packing Materials    

In [17]:
packing=height[height['buckets'].astype(str)=='Protective Packing Materials']
len(packing)#variable

1685

In [18]:

# not_tube[['length', 'width', 'height']]=not_tube['matches'].str.split("x",expand=True,)

# not_tube['length']=not_tube['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
# not_tube['width']=not_tube['width'].apply(lambda measure:   re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))

# not_tube['length']=not_tube['length'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))
# not_tube['width']=not_tube['width'].apply(lambda measure:   re.sub(r"\'|,",'',str(measure)))

# not_tube['length']=not_tube['length'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
# not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))

# not_tube['length']=not_tube['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
# not_tube['width']=not_tube['width'].apply(lambda quote: quote.replace('"', ''))

# regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
# not_tube['length_new']=not_tube['length'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
# not_tube['width_new']=not_tube['width'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)

# not_tube['length_new'] = not_tube['length_new'].apply(lambda x: remove_fraction(x))
# not_tube['width_new'] = not_tube['width_new'].apply(lambda x: remove_fraction(x))

# not_tube['length_new'] = not_tube['length_new'].apply(lambda x: remove_frac(x))
# not_tube['width_new'] = not_tube['width_new'].apply(lambda x: remove_frac(x))

# not_tube['length_new']=not_tube['length_new'].astype(float)
# not_tube['width_new']=not_tube['width_new'].astype(float)

# not_tube['Max'] = not_tube[['length_new', 'width_new']].values.max(1)
# not_tube['Min'] = not_tube[['length_new', 'width_new']].values.min(1)


# TWOPLACES = Decimal(10) ** -2
# not_tube['Max']=not_tube['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x)))
# not_tube['Min']=not_tube['Min'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

# envelope['envelope_matches']=envelope['envelope_matches'].str.get(0)
# envelope['envelope_matches']=envelope['envelope_matches'].apply(lambda x: x.lower())


# envelope[['length', 'width']]=envelope['envelope_matches'].str.split("x",expand=True,)

# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))

# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r"\'|,",'',str(measure)))

# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))

# envelope['length']=envelope['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
# envelope['width']=envelope['width'].apply(lambda quote: quote.replace('"', ''))
# # envelope['length']=envelope['length'].replace('"', '').apply(lambda quote: quote.replace('-',''))
# # envelope['width']=envelope['width'].apply(lambda quote: quote.replace('-',''))


# regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
# envelope['length_new']=envelope['length'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
# envelope['width_new']=envelope['width'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)


# envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_fraction(x))
# envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_fraction(x))

# envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_frac(x))
# envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_frac(x))

# Mailing & Packing Boxes

In [19]:
Boxes=height
Boxes=Boxes[Boxes['buckets'].astype(str)=='Mailing & Packing Boxes']
regex_pattern_tube=r'(?i)(tube)|()'
tubes=Boxes
tubes['tube_matches']=tubes['long_desc'].apply(lambda x: re_extract(regex_pattern_tube, x))

not_tube=tubes[tubes['tube_matches'].astype(str)=='[]']
regex_pattern=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|()'              
not_tube['matches']=not_tube['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
not_tube=not_tube[not_tube['matches'].astype(str)!='[]']

In [20]:
not_tube['matches']=not_tube['matches'].str.get(0).apply(lambda x: x.lower())
not_tube[['length', 'width', 'height']]=not_tube['matches'].str.split("x",expand=True,)

ValueError: Columns must be same length as key

In [ ]:
not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r'15250','15',str(measure)))
not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r'1232','12',str(measure)))
not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r'15250','15',str(measure)))
not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r'24141680','2',str(measure)))
not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r'16200','16',str(measure)))
not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r'9275','9',str(measure)))
not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r'1832','18',str(measure)))

In [ ]:
not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•','',str(measure)))
not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))

In [ ]:
not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
not_tube['width']=not_tube['width'].replace('"', '').apply(lambda quote: quote.replace('"', ''))

In [ ]:
regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
not_tube['width_new']=not_tube['width'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
not_tube['width_new'] = not_tube['width_new'].apply(lambda x: remove_fraction(x)) 

In [ ]:
not_tube['width_new'] = not_tube['width_new'].apply(lambda x: remove_frac(x)) 
not_tube['width_new']=not_tube['width_new'].astype(float) 
TWOPLACES = Decimal(10) ** -2
not_tube['width_new']=not_tube['width_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES))) 

In [ ]:
not_tube[curation_col]=not_tube['width_new']
not_tube[curation_col]=not_tube[curation_col].apply(lambda x:str(x)+' in')

In [ ]:
len(not_tube) 

# Envelope

In [35]:
envelope=height[height['buckets'].astype(str)=='Envelopes/Mailers']
regex_pattern_general=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*)|(\d+\.?\d*.?\s?x\s?\d+\.?\d*.?)|()'
envelope['envelope_matches']=envelope['product_name'].apply(lambda x: re_extract(regex_pattern_general, x))
envelope_na=envelope[envelope['envelope_matches'].astype(str)=='[]']

In [40]:
envelope_na[curation_col]='n/a'

In [ ]:
regex_pattern_general=r'(?i)(roll)|()'
envelope['envelope_matches']=envelope['product_name'].apply(lambda x: re_extract(regex_pattern_general, x))
envelope=envelope[envelope['envelope_matches'].astype(str)=='[]']
roll=envelope[envelope['envelope_matches'].astype(str)!='[]']
roll

In [22]:
regex_pattern=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|()'              
envelope['matches']=envelope['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
envelope=envelope[envelope['matches'].astype(str)=='[]']

In [23]:
# envelope[0:500] # Waiting on Arv and Ross

In [24]:
regex_pattern_general=r'(?i)(\d+\s?\-?\d+\/\d+.?\s?x\s?\d+\s?\-?\d+\/\d+)|(\d+\.\d*.?\s?x\s?\d+\.\d*)|(\d+\s?\-?\d+\/\d+.?\s?x\s?\d+\.?\d*)|(\d+\s?.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+)|(\d+.?\d*.?\s?x\s?\d+.?\d*)|(\d+\.?\s?\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|()'    
envelope['envelope_matches']=envelope['long_desc'].apply(lambda x: re_extract(regex_pattern_general, x))
envelope=envelope[envelope['envelope_matches'].astype(str)!='[]']

In [25]:
# def array(df):
#     df['matches_one']=df['envelope_matches'].apply(lambda x : np.array(x)[0])
#     try:
#         df['matches_two']=df['envelope_matches'].apply(lambda x : np.array(x)[1])
#     except:
#         df['matches_two']='n/a'

In [27]:
# def drop_characters(data):
#     data['length']=data['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))

In [28]:
# array(envelope)

In [29]:
# envelope['match_one'][4:6]=envelope['envelope_matches'][4:6].apply(lambda x : np.array(x)[0])

In [30]:
# try:
#     envelope['matches'][4:6]=envelope['envelope_matches'][4:6].apply(lambda x : np.array(x)[1])
# except:
#     envelope['matches'][4:6]=envelope['envelope_matches'][4:6].apply(lambda x : np.array(x)[0])

In [31]:
# envelope[0:500]

In [32]:
# # double=envelope[envelope['external_id'].astype(str)=='5637503643']
# double=envelope[0:1]
# double

In [33]:
# double['n_matches']=double['envelope_matches'].apply(lambda x : np.array(x)[0])
# # double['m_matches']=double['envelope_matches'].apply(lambda x : np.array(x)[1])
# double

In [34]:
# replacement={
#     '83/83':'1',    
# }

# # Replacement
# double= double.apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

In [35]:
# double=double.apply(lambda x: f'"{x}"')
# double=double.apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# double[['one','two']]=double['envelope_matches'].str.split(",",expand=True,)
# double=double.apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# double

In [36]:
envelope['envelope_matches']=envelope['envelope_matches'].str.get(0)
envelope['envelope_matches']=envelope['envelope_matches'].apply(lambda x: x.lower())

In [39]:
envelope

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, bucket_id, values, envelope_matches, matches]
Index: []

In [38]:
envelope[['width', 'length','height']]=envelope['envelope_matches'].str.split("x",expand=True,)

ValueError: Columns must be same length as key

In [33]:
envelope[envelope['width'].astype(str)=='77 34']

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, bucket_id, values, envelope_matches, matches, width, length]
Index: []

In [34]:
envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'77-34','34',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12 500\/','12',str(measure)))
envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'12 500\/','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12 2500\/','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12 1000\/','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12\,100','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'10 100/','10',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'8 100/','8',str(measure)))

envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12100','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12\,100','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'10 100/','10',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'8 100/','8',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'16 100','16',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'19 500','19',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'77-34','34',str(measure)))


In [35]:
envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))

envelope['length']=envelope['length'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r"\'|,",'',str(measure)))

In [36]:
envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))

envelope['length']=envelope['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
envelope['width']=envelope['width'].apply(lambda quote: quote.replace('"', ''))

regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
# envelope['length_new']=envelope['length'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
envelope['width_new']=envelope['width'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)

# envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_fraction(x))
envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_fraction(x))

# envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_frac(x))
envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_frac(x))

In [37]:
# envelope['length_new']=envelope['length_new'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
# envelope['width_new']=envelope['width_new'].apply(lambda quote: quote.replace('"', ''))

In [38]:
TWOPLACES = Decimal(10) ** -2
# envelope['length_new']=envelope['length_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
envelope['width_new']=envelope['width_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
envelope[curation_col]=envelope['width_new']

In [39]:
envelope[curation_col]=envelope[curation_col].apply(lambda x: str(x)+' in')

In [40]:
len(envelope)

5576

# Office Chairs

In [45]:
oc=height[height['buckets'].astype(str)=='Office Chairs']
# regex_pattern_general=r'(?i)(roll)|()'
regex_pattern=r'(?i)(?i)(\d+.?\d*\/?\d*.?\s?to\s?[a-z\s]{0,20}\d*.?\d*\/?\d*)|(seat.?width.?\s?\d+\"\s?\-?\s?\d*\s?\d*\/?\d*)|(\d\d\d?\d*\".{0,3}\d\d\.\d*\"\s?W)||()'              
oc['matches']=oc['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
oc_na=oc[oc['matches'].astype(str)=='[]']
oc=oc[oc['matches'].astype(str)!='[]']
oc_na[curation_col]=r'n/a'

In [46]:
len(oc_na) 

699

# Desk WorkBenches

In [139]:
desk=height[height['buckets'].astype(str)=='Desks/Workstations/Workbenches']
# regex_pattern_general=r'(?i)(roll)|()'
regex_pattern=r'(?i)(\d+.?w\s?x\s?30)|(\d+\.?\d*.?w\s?x\s?30)|(\d+\s?\-?\/?\d+.?w\s?x\s?30)|(\d+.?\s?x\s?30)|(\d+\.?\d*.?\s?x\s?30)|(\d+\-?\/?\d+.?\s?x\s?30)|(width)|(\"w)|(\d\d\.?\d\s?w)|(\d+\s?\d*\/\d*\s?w)|(\d+.?\s?x\s?\d*.?\d*\/\d+)|(\d+.?.?w?\s?x\s?\d+.?\d*.?)|(\d+.?\s?w\s?)|()'              
desk['matches']=desk['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
desk_na=desk[desk['matches'].astype(str)=='[]']
desk=desk[desk['matches'].astype(str)!='[]']
desk_na[curation_col]=r'n/a'

In [156]:
len(desk) 

639

In [150]:
desk['one']=desk['matches'][7:9].str.get(0)
desk['two']=desk['matches'][7:9].str.get(1)
desk['three']=desk['matches'][7:9].str.get(2)
desk['four']=desk['matches'][7:9].str.get(3)

In [159]:
desk['two'][0:1].astype(str)

37    nan
Name: two, dtype: object

In [163]:
desk[desk['two'].astype(str)!='nan']

product_id external_id  \
360  1175394879  5637576133   

                                                                       product_name  \
360  Flash Furniture Clifton Maple Computer Desk With Top And Lower Storage Shelves   

                                                                                                                                                                  long_desc  \
360  Clifton Maple Computer Desk With Top And Lower Storage Shelves•Contemporary style•Maple laminate finish•Top size: 28"W x 14"D•.5" thick top•Top shelf size: 28"W x 8"D   

     customer_id attributes                         buckets  bucket_id values  \
360           90       None  Desks/Workstations/Workbenches       7440   None   

                       matches         one          two  three  four  
360  [28"W x 8"D, 28"W x 14"D]  28"W x 8"D  28"W x 14"D    NaN   NaN

# Chairs

In [169]:
chair=height[height['buckets'].astype(str)=='Chairs']
regex_pattern=r'(?i)(\d+\.?\s?\-?\d*\/?\d*\"\s?w)|()'              
chair['matches']=chair['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
chair_height=chair[chair['matches'].astype(str)!='[]']

In [179]:
print(len(chair))
print(len(chair_height))

692
78


In [175]:
chair_three=chair[chair['matches'].astype(str)=='[]']
regex_pattern_three=r'(?i)(\d+.?w\s?x\s?30)|(\d+\.?\d*.?w\s?x\s?30)|(\d+\s?\-?\/?\d+.?w\s?x\s?30)|(\d+.?\s?x\s?30)|(\d+\.?\d*.?\s?x\s?30)|(\d+\-?\/?\d+.?\s?x\s?30)|(width)|(\"w)|(\d\d\.?\d\s?w)|(\d+\s?\d*\/\d*\s?w)|(\d+.?\s?x\s?\d*.?\d*\/\d+)|(\d+.?.?w?\s?x\s?\d+.?\d*.?)|(\d+.?\s?w\s?)|()'              
chair_three['matches']=chair_three['long_desc'].apply(lambda x: re_extract(regex_pattern_three, x))

In [176]:
chair_na=chair_three[chair_three['matches'].astype(str)=='[]']
chair_na[curation_col]=r'n/a'

In [177]:
print(len(chair_na)) 

584


# Drawers

In [186]:
drawer=height[height['buckets'].astype(str)=='Drawers']
print(len(drawer))
regex_pattern_drawer=r'(?i)(\d\d\.?\d*\s?w)|(\d+\s?\d*\/\d*\s?w)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?W)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|(\d+.?\d*\"\s?To\s?\d+")|(\d+\.?\s?\-?\d*\/?\d*"W)|(width)|()'                           
drawer['matches']=drawer['long_desc'].apply(lambda x: re_extract(regex_pattern_drawer, x))
drawer_na=drawer[drawer['matches'].astype(str)=='[]']
drawer_na[curation_col]=r'n/a'
print(len(drawer_na))

456
137


In [187]:
drawer_next=drawer[drawer['matches'].astype(str)!='[]']
regex_pattern_short=r'(?i)(\d\d\.?\d*\s?w)|(\d+\s?\-?\d*\/\d*\s?w)|(\d+\.?\s?\-?\d*\/?\d*"W)|()'                           
drawer_next['new_matches']=drawer_next['long_desc'].apply(lambda x: re_extract(regex_pattern_short, x))
drawer_short=drawer_next[drawer_next['new_matches'].astype(str)!='[]']
print(len(drawer_short))

293


In [192]:
drawer_short[drawer_short['width'].astype(str)=='500 16']

product_id external_id  \
14709  1175425699  5637577971   

                                                                               product_name  \
14709  Bush Business Furniture Office 500 16"W 3-Drawer File Cabinet, Storm Gray, Assembled   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    long_desc  \
14709  Office 500 16"W 3-Drawer File Cabinet, Storm Gray, Assembled•16"W x 20"D x 29"H 3-Drawer File Cabinet ships fully assembled for a stress-free set up•Drawers fit under Office 500 72"W x 36"D Executive Desk and 72"W x 24"D Credenza Desk (sold separately)•File drawer opens on smooth full-extension ball bearing slides for easy access to legal, letter and A4-size paperwork•Vertical filing cabinet also includes two box drawers for organizing small office supplies•Single lock secures two bottom drawers to protect sensitive documents and personal items   

       customer_id attributes  buckets  bucket_id values  \
14709           90       None  Drawers       7476   None   

                                matches                       new_matches  \
14709  [500 16"W, 72"W, 500 72"W, 16"W]  [500 16"W, 72"W, 500 72"W, 16"W]   

        width  
14709  500 16

In [209]:
drawer_short['width']=drawer_short['new_matches'].str.get(0)
drawer_short['width']=drawer_short['width'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
drawer_short['width']=drawer_short['width'].apply(lambda measure: re.sub(r'500 36','36',str(measure)))
drawer_short['width']=drawer_short['width'].apply(lambda measure: re.sub(r'500 16','16',str(measure)))
drawer_short['width']=drawer_short['width'].apply(lambda measure: re.sub(r"\'|,|:",'',str(measure)))
drawer_short['width']=drawer_short['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
drawer_short['width']=drawer_short['width'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
drawer_short['width'] = drawer_short['width'].apply(lambda x: remove_fraction(x))
drawer_short['width'] = drawer_short['width'].apply(lambda x: remove_frac(x))
drawer_short['width']=drawer_short['width'].astype(float)
TWOPLACES = Decimal(10) ** -2
drawer_short['width']=drawer_short['width'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
drawer_short[curation_col]=drawer_short['width']
drawer_short[curation_col]=drawer_short[curation_col].apply(lambda x: str(x)+' in')

In [196]:
drawer_three=drawer_next[drawer_next['new_matches'].astype(str)=='[]']
regex_pattern_three=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?W)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|()'
drawer_three['new_matches']=drawer_three['long_desc'].apply(lambda x: re_extract(regex_pattern_three, x))
drawer_three=drawer_three[drawer_three['new_matches'].astype(str)!='[]']

In [197]:
drawer_three['new']=drawer_three['new_matches'].str.get(0)
drawer_three[['length', 'width', 'height']]=drawer_three['new'].str.split("x",expand=True,)
drawer_three=drawer_three[(drawer_three['external_id'].astype(str)!='5637192984')&(drawer_three['external_id'].astype(str)!='5637192989')&(drawer_three['external_id'].astype(str)!='5637192983')&(drawer_three['external_id'].astype(str)!='5637192989')&(drawer_three['external_id'].astype(str)!='5637192990')]                      

In [198]:
drawer_three['width']=drawer_three['width'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
drawer_three['width']=drawer_three['width'].apply(lambda measure: re.sub(r"\'|,|:",'',str(measure)))
drawer_three['width']=drawer_three['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
drawer_three['width']=drawer_three['width'].replace('"', '').apply(lambda quote: quote.replace('"', ''))

In [199]:
regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
drawer_three['width_new']=drawer_three['width'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
drawer_three['width_new'] = drawer_three['width_new'].apply(lambda x: remove_fraction(x))

In [208]:
drawer_three['width_new']=drawer_three['width_new'].apply(lambda x: remove_fraction(x))
drawer_three['width_new']=drawer_three['width_new'].apply(lambda x: remove_frac(x))
drawer_three['width_new']=drawer_three['width_new'].astype(float)
TWOPLACES = Decimal(10) ** -2
drawer_three['width_new']=drawer_three['width_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
drawer_three[curation_col]=drawer_three['width_new']
drawer_three[curation_col]=drawer_three[curation_col].apply(lambda x: str(x)+' in')

In [210]:
print(len(drawer_short))

293


In [211]:
print(len(drawer_three))

22


In [212]:
drawer_three

product_id external_id  \
1054    477458063  5637506399   
1062    434387388  5637218263   
1125    434385116  5637216865   
1654    434385211  5637218880   
2526    434407631  5637306801   
4406    434485770  5637149753   
4407    434485769  5637149751   
8392    434451221  5637408078   
8393    434452026  5637408060   
8609    477457460  5637507300   
8790    477458060  5637506397   
9360    434451223  5637408080   
12686   477458064  5637506400   
12904   477458032  5637506376   
12956   477458033  5637506377   
13115   434451224  5637408081   
13236   474986760  5637506693   
13402   477458031  5637506375   
15071   434451222  5637408079   
15072   434452027  5637408061   
16979   477458061  5637506398   
17205   474986744  5637506682   

                                                                                               product_name  \
1054                              Lorell Revelance Mobile Pedestal, 15.8" x 19.9" x 22.9", Laminate, Walnut   
1062                  Safco Mira Series Wood Veneer Two-Drawer Lateral File, 34-1/2 x 24 x 29-1/2, Espresso   
1125                  Storex Two-Drawer Mobile Filing Cabinet With Metal Siding, 19 x 15 x 23, Silver/Black   
1654                  Safco Mira Series Wood Veneer Two-Drawer Lateral File, 34-1/2 x 24 x 29-1/2, Med Chry   
2526                    HON BL Laminate Three Drawer Pedestal File, 15 5/8 x 21 3/4 x 27 3/4, Medium Cherry   
4406                              HON BW Veneer Series Two-Drawer Lateral File, 36 x 24 x 29,Bourbon Cherry   
4407                     HON BW Veneer Series File/File Pedestal File, 15-5/8 x 22 x 27-3/4, Bourbon Cherry   
8392                                            HON Mobile Pedestal File, Box/Box/File, 15 x 20 x 28, Putty   
8393                                            HON Mobile Pedestal File, Box/Box/File, 15 x 20 x 28, Black   
8609                          Lorell 3-drawer Mobile Pedestal File, 15" x 19" x 26", Steel, Recycled, White   
8790            Lorell Relevance Series Box/File Mobile Pedestal, 15.8" x 19.9" x 22.9", Laminate, Mahogany   
9360                                               HON Mobile Pedestal File, File/File, 15 x 20 x 28, Black   
12686                      Lorell Mid-century Modern Box/File Mobile Pedestal, 19.8" x 16.5" x 23.5", White   
12904  Lorell Relevance Series Office File Cabinet Credenza, 29.5" x 22" x 23.1", Laminate, Charcoal/Silver   
12956                        Lorell Open Shelf File Cabinet Credenza, 29.5" x 22" x 23.1", Laminate, Walnut   
13115                                               HON Mobile Pedestal File, File/File, 15 x 20 x 28, Gray   
13236                                     HON 4-Drawer Lateral File, 36" x 19.3" x 53.3", Steel, Light Gray   
13402         Lorell Relevance Series Office File Cabinet Credenza, 29.5" x 22" x 23.1", Laminate, Mahogany   
15071                                              HON Mobile Pedestal File, File/File, 15 x 20 x 28, Putty   
15072                                            HON Mobile Pedestal File, Box/Box/File, 15 x 20 x 28, Gray   
16979           Lorell Relevance Series Box/File Mobile Pedestal, 15.8" x 19.9" x 22.9", Laminate, Charcoal   
17205                                     HON 2-Drawer Lateral File, 36" x 19.8" x 28.4", Steel, Light Gray   

                                                                                                                                                                                                                                                                                                                                                                                                            long_desc  \
1054                                                                                              Revelance Mobile Pedestal, 15.8" x 19.9" x 22.9", Laminate, Walnut•Part of the Lorell Relevance Series•Mobile for easy access; 2 casters lock for reliable stability•Versatile storage wit

In [204]:
print(len(drawer_na))

137


In [205]:
print(len(chair_na)) 

584


In [143]:
chair_na

product_id external_id  \
13      492030428  5637508245   
15      434513539  5637490727   
16      434442195  5637389544   
17      434485502  5637171439   
18      434501924  5637476114   
...           ...         ...   
17625   815179205  5637541635   
17633   815177544  5637541618   
17641   815179215  5637541644   
17656   815176118  5637541626   
17658   815179200  5637541671   

                                                                                                        product_name  \
13                                                     Flash Furniture HERCULES Diplomat Series Chair, Leather, Gray   
15                     Flash Furniture HERCULES Series 661 lb. Capacity Black Full Back Stack Chair with Black Frame   
16                                                          HON Accommodate� Series Armless Guest Chair, Black Vinyl   
17                                                         Safco Uber Series Big & Tall Sled Base Guest Chair, Black   
18                                    Allsteel Lyric Multi-Purpose Chair, Polymer Back, Fixed Arms, Fixed Arms, Lava   
...                                                                                                              ...   
17625  Flash Furniture HERCULES Series Trapezoidal Back Stacking Banquet Chair in Navy Patterned Fabric - Gold Frame   
17633                   Flash Furniture HERCULES Series 21''W Stacking Church Chair in Plum Fabric - Gold Vein Frame   
17641                             Flash Furniture HERCULES Series Big & Tall 1000 lb. Rated Black Fabric Stack Chair   
17656  Flash Furniture HERCULES Series 21''W Church Chair in Navy Blue Fabric with Cup Book Rack - Silver Vein Frame   
17658                            Flash Furniture White LeatherSoft Executive Side Reception Chair with Mahogany Legs   

                                                                                                                                                                                                                                                                                                                                                   long_desc  \
13                                                                                                                                                                                                 HERCULES Diplomat Series Chair, Leather, Gray•Contemporary style•Gray LeatherSoft upholstery•Flared arms•Plush back and seat•Fixed back and seat cushions   
15                                                                                                                                                    HERCULES Series 661 lb. Capacity Black Full Back Stack Chair with Black Frame•Multipurpose stack chair•661 lb. weight capacity•Stack quantity: 25•Plastic back and seat•Ergonomically contoured design   
16                Accommodate� Series Armless Guest Chair, Black Vinyl•Lightweight, maneuverable frame is ideal for multi-purpose spaces, cafes and common areas.•Seat cushion is made of high-density molded foam that is designed to last longer.•Comfort contours in the seat back cradle the spine.•Waterfall seat edge promotes better leg circulation.   
17                                                                                                              Uber Series Big & Tall Sled Base Guest Chair, Black•Specifically designed for the tall and big person.•Extra-wide and deep seat.•Cushioned foam is rated for 24/7 use and chair is rated up to 500 lbs.•Reinforced base for added stability.   
18     Lyric Multi-Purpose Chair, Polymer Back, Fixed Arms, Fixed Arms, Lava•Perforated back provides ventilation and distributed support•Waterfall seat front reduces pressure points and the intuitive back design offers built-in lumbar support•Available with or without upholstered seat•Stack up to 10 chairs high•Warranted for users up to 300 lbs.   
...                                                      

In [64]:
len(desk_na) 

285

In [65]:
len(oc_na) 

693

In [66]:
len(envelope)

5578

In [57]:
err

NameError: name 'err' is not defined

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

In [ ]:
len(envelope)len(oc_na) len(desk_na) print(len(chair_na)) print(len(drawer_na)) print(len(drawer_three)) print(len(drawer_short))

# send to the folder for upload

In [20]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets- {attribute}-- tubes_new- upload_Shawn.csv',index=False)

In [21]:
looks_good('Bed Bath & Beyond', attribute, df, tubes_new)

In [59]:
def looks_good_na(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets-NA - {attribute}-- total_na_values upload_Shawn.csv',index=False)

In [60]:
looks_good_na('Bed Bath & Beyond', attribute, df, total_na_values)